In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from matplotlib.ticker import NullFormatter


# read csv
csv = "Billionaire_Data.csv"
data_df = pd.read_csv(csv)

In [ ]:
data_df["gdp_country"] = data_df["gdp_country"].str.replace("$", "")
data_df["gdp_country"] = data_df["gdp_country"].str.replace(",", "")
data_df["gdp_country"]=data_df["gdp_country"].astype(float)

In [ ]:
data_df["finalWorth"]/=1000
data_df['gdp_country']/=1000000000
data_df.head()